In [1]:
// import Spark from Maven (dependency manager) repository
import $ivy.`org.apache.spark::spark-sql:3.0.0`;

import $ivy.$                                  ;

In [2]:
// Disable logs to avoid clogging up cell output
// RECOMMEND enabling for debugging purposes
import org.apache.log4j.{Level, Logger};
Logger.getLogger("org").setLevel(Level.OFF);

import org.apache.log4j.{Level, Logger};


In [3]:
import org.apache.spark.sql._

val spark = SparkSession.
            builder().
            appName("scala-spark-notebook").
            master("spark://spark-master:7077").
            config("spark.executor.memory", "512m").
            .setJars(new String[]{"/path/to/jar/with/your/class.jar"}).
            getOrCreate()

val sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@548561bd
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4b47fc36

In [5]:
import org.apache.spark

val SEP = ","

// Does not work for some reason..
def removeHeader(rdd: spark.rdd.RDD[String]): spark.rdd.RDD[String] = {
    val header = rdd.first()
    return rdd.filter(line => line != header)
}

// Load data and drop header
// val rawData = sc.textFile("/data/airports_data.csv")

// None work
// val airportsData = rawData.filter(line  => !line.contains("airport"))
// val airportsData = removeHeader( rawData )
// val airportsData = sc.textFile("/data/airports_data2.csv")

// Extract KV pair using function
def extractTimezoneToLatLong(line: String): (String, (Float, Float)) = {
    val nsplit = line.split(SEP)
    return (nsplit(5), (nsplit(4).toFloat, nsplit(3).toFloat)) // Scala Lists are indexed as list(i)
}
val tzLatLong = airportsData.map(extractTimezoneToLatLong)

// Double the Lat and Long using a lambda function
val doubleLatLong = (tup: (String, (Float, Float))) => (tup._1, (2*tup._2._1, 2*tup._2._2))
val tzLatLongDbl = tzLatLong.map(doubleLatLong) 

// Aggregate using inline lambda
val totalByTz = tzLatLongDbl.reduceByKey(((v1, v2) => (v1._1 + v2._1, v1._2 + v2._2)))

import org.apache.spark


SEP: String = ","
airportsData: spark.rdd.RDD[String] = /data/airports_data2.csv MapPartitionsRDD[3] at textFile at cmd4.sc:13
defined function extractTimezoneToLatLong
tzLatLong: spark.rdd.RDD[(String, (Float, Float))] = MapPartitionsRDD[4] at map at cmd4.sc:22
doubleLatLong: (String, (Float, Float)) => (String, (Float, Float)) = ammonite.$sess.cmd4$Helper$$Lambda$3734/2065157289@3080dbab
tzLatLongDbl: spark.rdd.RDD[(String, (Float, Float))] = MapPartitionsRDD[5] at map at cmd4.sc:26
totalByTz: spark.rdd.RDD[(String, (Float, Float))] = ShuffledRDD[6] at reduceByKey at cmd4.sc:29

In [6]:
totalByTz.collect()

: 